In [1]:
import requests
import ipykernel
import re
from notebook.notebookapp import list_running_servers
from notebook import notebookapp

servers = list(notebookapp.list_running_servers())

TOKEN = servers[0]['token']

base_url = next(list_running_servers())['url']
r = requests.get(
    url=base_url + 'api/sessions',
    headers={'Authorization': 'token {}'.format(TOKEN),})

r.raise_for_status()
response = r.json()

kernel_id = re.search(
    'kernel-(.*).json',
    ipykernel.connect.get_connection_file()
).group(1)

NOTEBOOK_PATH = {
    r['kernel']['id']: r['notebook']['path']
    for r in response
}[kernel_id]

In [2]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from itertools import chain
from collections import Counter
import unidecode

import glob
import re

from dodfminer.extract.polished.acts.aposentadoria import Retirements
from drawBoxes.drawBoxes import draw2

import fitz
from utils import get_dodf_num
from utils import get_dodf_tipo
from utils import get_dodf_reverse_date
from utils import get_dodf_key

import pickle

In [3]:
def get_doc_blocks(doc):
    lis = []
    for idx, page in enumerate(doc, start=1):
        bls = page.getTextBlocks()
        bls = [(*i, idx) for i in bls]
        lis.extend(bls)
    return lis

def get_dodfs_path():
    return glob.glob('data/aposentadoria-ouro/pdfs/*.pdf')

def wRetirements(s):
    return Retirements(None, 'regex',
                       txt= unidecode.unidecode_expect_ascii(s))
def f(x):
    if x.name == 'data':
        return pd.to_datetime(x)
    elif x.name == 'num':
        return x.astype(np.int16)
    return x

INDEX_COL = ['data', 'num', 'tipo']

NUM_FILES = 3
NUM_FILES = None
PATHS = get_dodfs_path()[:NUM_FILES]

In [4]:
%%time
df = pd.DataFrame()
paths = get_dodfs_path()
aposentadorias = {get_dodf_key(i): wRetirements('\n'.join(
        [i[4] for i in get_doc_blocks(fitz.open(i))]
    ))
    for i in paths
}

CPU times: user 2min 20s, sys: 1.62 s, total: 2min 21s
Wall time: 2min 21s


In [5]:
ks, vals = list(aposentadorias.keys()), list(aposentadorias.values())

In [6]:
ks[0]

('2019-08-16', 155, 'NORMAL')

In [7]:
df = pd.DataFrame.from_records(
    [ (*k, v) for (k,v) in aposentadorias.items() ],
    columns=INDEX_COL + ['qtd_miner'],
)
df['qtd_miner'] = df.qtd_miner.map(lambda x: x.data_frame.shape[0])
df.set_index(INDEX_COL, inplace=True)
df.sort_index(key=f, inplace=True)
df.head(3)

,,,qtd_miner
data,num,tipo,
2018-04-02,62,NORMAL,10
2018-04-03,63,NORMAL,1
2018-04-04,64,NORMAL,15


In [8]:
prf = 'dminer_ctr'
df.to_csv(f'{prf}.csv')
open(f'{prf}.txt', 'w').write(
    f"notebook de origem: {NOTEBOOK_PATH}\n\n"
    f"O csv correspondente ({prf}.csv) contém o contador de quantos\n"
    "casos de aposentadoria foram detectados pela classe Retirements \n"
    "considerando os textos dos blocos de todas as páginas concatenados\n"
    "separados por quebra de linha."
)
del prf